# No data? No problem! Genera datasets sintéticos con Python

**PyCon España 2025 - Workshop (90 minutos)**

## Objetivos del workshop:
- ✅ Generar datos sintéticos realistas desde cero
- ✅ Controlar el resultado con filtros y condiciones específicas  
- ✅ Crear conjuntos multitabla con relaciones entre entidades
- ✅ Evaluar la calidad y utilidad de los datos sintéticos
- ✅ Trabajar con datos sin comprometer la privacidad

**Dataset**: US Census Income  
**SDK**: MostlyAI (open source)

---


## 0. Setup (2 min)

Instalamos el SDK y configuramos el entorno.


In [ ]:
%pip install -U "mostlyai[local]" -q


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mostlyai.sdk import MostlyAI

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)

mostly = MostlyAI(local=True)
np.random.seed(42)

print("✅ Setup completo!")


---

## 1. El Problema: Reidentificación (12 min)

### 🎯 Escenario
Trabajas en RRHH y necesitas compartir datos salariales con una consultora para un análisis de brecha salarial.

Eliminas los nombres y piensas: *"Ya está anonimizado"*

**¿Es suficiente?** 🤔


### Cargar el dataset Census


In [ ]:
url = "https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz"
census = pd.read_csv(url)

print(f"Dataset: {census.shape[0]:,} registros, {census.shape[1]} columnas")
print(f"\nColumnas: {list(census.columns)}")
census.head()


In [ ]:
census.describe()


### Demo de Reidentificación

Imagina que conoces a alguien con estas características:
- Mujer
- PhD (Doctorado)
- Ejecutiva
- 45-50 años
- Gana >50K

**¿Podemos encontrarla en el dataset "anonimizado"?**


In [ ]:
print("🔍 DEMO: REIDENTIFICACIÓN")
print("="*60)

candidatos = census[
    (census['sex'] == 'Female') &
    (census['education'] == 'Doctorate') &
    (census['age'] >= 45) & (census['age'] <= 50) &
    (census['occupation'] == 'Exec-managerial') &
    (census['income'] == '>50K')
]

print(f"\nBuscando: Mujer, PhD, ejecutiva, 45-50 años, >50K")
print(f"Candidatos encontrados: {len(candidatos)}")

if len(candidatos) <= 3:
    print("\n⚠️  ¡FÁCILMENTE IDENTIFICABLE!")
    print("Con información pública (LinkedIn, redes) → identificación completa")
    if len(candidatos) > 0:
        print("\nEjemplo de registro identificable:")
        print(candidatos.iloc[0][['age', 'education', 'occupation', 'hours_per_week', 'income']])


### Análisis de riesgo general


In [ ]:
print("📊 ANÁLISIS DE RIESGO GENERAL")
print("="*60)

combos = census.groupby(['sex', 'education', 'occupation', 'age']).size()
unicos = combos[combos == 1]

print(f"\nCombinaciones únicas: {len(unicos):,} de {len(census):,}")
print(f"Porcentaje de registros únicos: {len(unicos)/len(census)*100:.1f}%")

print("\n🚨 CONCLUSIÓN:")
print("   • Miles de personas son identificables con solo 4 características")
print("   • Anonimización tradicional NO funciona")
print("   • Necesitamos una solución mejor: DATOS SINTÉTICOS")


### Casos reales de reidentificación

**Ejemplos históricos:**
- **AOL Search Data (2006)**: 650,000 usuarios "anonimizados" → periodistas identificaron individuos por sus búsquedas
- **Netflix Prize (2007)**: Dataset "anonimizado" → investigadores reidentificaron usuarios cruzando con IMDB
- **NYC Taxi Data (2014)**: Viajes "anonimizados" → identificaron celebridades y sus destinos

**La solución: Datos Sintéticos**
- ✅ Privacidad real (no contienen información de personas reales)
- ✅ Utilidad preservada (mantienen patrones estadísticos)
- ✅ Sin riesgo de reidentificación
- ✅ Compartibles libremente

---

## 2. Fundamentos: Tu primer dataset sintético (18 min)

Vamos a generar una versión sintética del Census dataset que:
1. Preserve las distribuciones estadísticas
2. No contenga información de personas reales
3. Sea útil para análisis y ML

### Preparar los datos

In [ ]:
from sklearn.model_selection import train_test_split

census_sample = census.sample(10000, random_state=42)

train_data, holdout_data = train_test_split(
    census_sample, 
    test_size=0.2, 
    random_state=42,
    stratify=census_sample['income']
)

print(f"Training: {len(train_data):,} registros")
print(f"Holdout: {len(holdout_data):,} registros")

### Entrenar el generator

El generator aprende los patrones estadísticos del dataset original.

In [ ]:
config = {
    'name': 'Census Income Generator',
    'tables': [{
        'name': 'census',
        'data': train_data,
        'tabular_model_configuration': {
            'max_training_time': 3
        }
    }]
}

print("🚀 Entrenando generator...")
print("(Esto puede tardar unos minutos)\n")

g = mostly.train(config=config)

print("\n✅ Generator entrenado!")

### Generar datos sintéticos

Primero probamos con pocos registros, luego generamos el dataset completo.

In [ ]:
probe = mostly.probe(g, size=5)
print("🔬 Probe (5 registros sintéticos):")
probe

In [ ]:
print("🎲 Generando dataset sintético completo...\n")

syn_dataset = mostly.generate(g, size=len(train_data))
syn_data = syn_dataset.data()

print(f"\n✅ Dataset sintético: {syn_data.shape[0]:,} registros")
syn_data.head()

### Comparar distribuciones: Real vs Sintético

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

cols_to_plot = ['age', 'education', 'occupation', 'income', 'hours_per_week', 'sex']

for idx, col in enumerate(cols_to_plot):
    ax = axes[idx]
    
    if train_data[col].dtype == 'object' or train_data[col].nunique() < 10:
        real_counts = train_data[col].value_counts(normalize=True).sort_index()
        syn_counts = syn_data[col].value_counts(normalize=True).sort_index()
        
        x = np.arange(len(real_counts))
        width = 0.35
        
        ax.bar(x - width/2, real_counts.values, width, label='Real', alpha=0.8)
        ax.bar(x + width/2, syn_counts.values, width, label='Sintético', alpha=0.8)
        ax.set_xticks(x)
        ax.set_xticklabels(real_counts.index, rotation=45, ha='right', fontsize=8)
    else:
        ax.hist(train_data[col].dropna(), bins=30, alpha=0.6, label='Real', density=True)
        ax.hist(syn_data[col].dropna(), bins=30, alpha=0.6, label='Sintético', density=True)
    
    ax.set_title(col, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Las distribuciones se preservan!")

### Verificar: ¿Funciona la reidentificación en datos sintéticos?

In [ ]:
print("🔍 VERIFICACIÓN: Reidentificación en datos SINTÉTICOS")
print("="*60)

candidatos_syn = syn_data[
    (syn_data['sex'] == 'Female') &
    (syn_data['education'] == 'Doctorate') &
    (syn_data['age'] >= 45) & (syn_data['age'] <= 50) &
    (syn_data['occupation'] == 'Exec-managerial') &
    (syn_data['income'] == '>50K')
]

print(f"\nCandidatos en datos REALES: {len(candidatos)}")
print(f"Candidatos en datos SINTÉTICOS: {len(candidatos_syn)}")

combos_syn = syn_data.groupby(['sex', 'education', 'occupation', 'age']).size()
unicos_syn = combos_syn[combos_syn == 1]

print(f"\nCombinaciones únicas en REALES: {len(unicos):,} ({len(unicos)/len(train_data)*100:.1f}%)")
print(f"Combinaciones únicas en SINTÉTICOS: {len(unicos_syn):,} ({len(unicos_syn)/len(syn_data)*100:.1f}%)")

print("\n✅ CONCLUSIÓN: Los datos sintéticos NO permiten reidentificación!")

---

## 3. Evaluación: ¿Son buenos estos datos? (12 min)

Dos formas de evaluar calidad:
1. **QA Report**: Métricas automáticas de fidelidad
2. **TSTR**: Train-on-Synthetic / Test-on-Real

### QA Report automático

In [ ]:
print("📊 Quality Assurance Report:\n")
g.reports(display=True)

**Métricas del QA Report:**

- **Accuracy**: Compara distribuciones univariadas y bivariadas
- **Similarity**: Compara embeddings de orden superior (patrones complejos)
- **Distance**: DCR (Distance to Closest Record) - mide novelty

### TSTR: Train-on-Synthetic / Test-on-Real

**Pregunta**: ¿Puedo entrenar un modelo con datos sintéticos y que funcione en datos reales?

**Tarea**: Predecir income (>50K vs <=50K)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

numeric_cols = ['age', 'capital_gain', 'capital_loss', 'hours_per_week', 'fnlwgt']
categorical_cols = ['workclass', 'education', 'marital_status', 'occupation', 
                    'relationship', 'race', 'sex', 'native_country']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

def evaluate_tstr(train_df, test_df, label):
    train_clean = train_df.dropna()
    test_clean = test_df.dropna()
    
    X_train = train_clean.drop(columns=['income'])
    y_train = train_clean['income']
    
    X_test = test_clean.drop(columns=['income'])
    y_test = test_clean['income']
    
    clf = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=500))
    ])
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print(f"\n{'='*60}")
    print(f"{label}")
    print(f"{'='*60}")
    print(classification_report(y_test, y_pred, digits=3))
    
    return classification_report(y_test, y_pred, output_dict=True)

report_tstr = evaluate_tstr(syn_data, holdout_data, "TSTR: Train-on-Synthetic / Test-on-Real")
report_real = evaluate_tstr(train_data, holdout_data, "Baseline: Train-on-Real / Test-on-Real")

### Comparación visual

In [ ]:
metrics = ['precision', 'recall', 'f1-score']
tstr_scores = [report_tstr['weighted avg'][m] for m in metrics]
real_scores = [report_real['weighted avg'][m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, tstr_scores, width, label='Train-on-Synthetic', alpha=0.8)
ax.bar(x + width/2, real_scores, width, label='Train-on-Real', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('Comparación TSTR vs Train-on-Real', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.set_ylim(0.6, 0.9)
ax.legend()
ax.grid(axis='y', alpha=0.3)

for i, (tstr, real) in enumerate(zip(tstr_scores, real_scores)):
    ax.text(i - width/2, tstr + 0.01, f'{tstr:.3f}', ha='center', fontsize=10)
    ax.text(i + width/2, real + 0.01, f'{real:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n✅ Los datos sintéticos son útiles para ML!")

---

## 4. Control del resultado: Filtros y condiciones (20 min)

Ahora que sabemos generar datos sintéticos, vamos a controlar QUÉ generamos.

### A. Seeded Generation (Generación Condicional)

**Caso de uso**: Quiero generar solo mujeres con PhD

**Cómo funciona**: Fijamos algunas columnas, el generator "completa" el resto manteniendo coherencia.

In [ ]:
print("🎯 SEEDED GENERATION: Solo mujeres con PhD\n")

seed_women_phd = pd.DataFrame({
    'sex': ['Female'] * 20,
    'education': ['Doctorate'] * 20
})

print("Seed data (lo que fijamos):")
print(seed_women_phd.head())

syn_conditional = mostly.generate(g, seed=seed_women_phd)
syn_women_phd = syn_conditional.data()

print("\nDatos sintéticos generados (el generator completó el resto):")
print(syn_women_phd[['sex', 'education', 'age', 'occupation', 'income', 'hours_per_week']].head(10))

print(f"\n✅ Verificación:")
print(f"   • Todas mujeres: {(syn_women_phd['sex'] == 'Female').all()}")
print(f"   • Todas PhD: {(syn_women_phd['education'] == 'Doctorate').all()}")
print(f"   • Ocupaciones variadas: {syn_women_phd['occupation'].nunique()} diferentes")
print(f"   • Edades variadas: {syn_women_phd['age'].min():.0f} - {syn_women_phd['age'].max():.0f} años")

### Ejemplo 2: Generar personas jóvenes con altos ingresos

In [ ]:
print("🎯 SEEDED GENERATION: Jóvenes con altos ingresos\n")

seed_young_rich = pd.DataFrame({
    'age': [25, 26, 27, 28, 29, 30] * 5,
    'income': ['>50K'] * 30
})

syn_young_rich = mostly.generate(g, seed=seed_young_rich).data()

print("Datos sintéticos generados:")
print(syn_young_rich[['age', 'income', 'education', 'occupation', 'hours_per_week']].head(10))

print(f"\n📊 Distribución de educación en jóvenes con >50K:")
print(syn_young_rich['education'].value_counts())

### 💡 Momento interactivo

**¿Qué otras condiciones queréis probar?**

Ideas:
- Solo personas con Masters trabajando en Tech
- Personas mayores de 60 años
- Personas con ingresos bajos pero alta educación

Probad vosotros:

In [ ]:
# Tu turno: crea tu propia seed data

# my_seed = pd.DataFrame({
#     # Añade tus condiciones aquí
# })

# syn_custom = mostly.generate(g, seed=my_seed).data()
# print(syn_custom.head())

### B. Imputation Inteligente

**Caso de uso**: Tengo datos con valores faltantes en `education`

**Solución**: El generator imputa valores coherentes basándose en el resto de características

In [ ]:
print("🔧 IMPUTATION: Completar valores faltantes\n")

data_with_missing = train_data[train_data['education'].isnull()].copy()

if len(data_with_missing) == 0:
    print("No hay valores faltantes en education. Creando algunos artificialmente...")
    data_with_missing = train_data.sample(50, random_state=42).copy()
    data_with_missing['education'] = None

print(f"Registros con education faltante: {len(data_with_missing)}")
print("\nEjemplo de datos con valores faltantes:")
print(data_with_missing[['age', 'occupation', 'education', 'income']].head())

imputation_config = {
    'tables': [{
        'configuration': {
            'imputation': {'columns': ['education']}
        }
    }]
}

syn_imputed = mostly.generate(g, config=imputation_config, seed=data_with_missing).data()

print("\n✅ Datos con education imputada:")
print(syn_imputed[['age', 'occupation', 'education', 'income']].head())

print(f"\nValores faltantes después de imputation: {syn_imputed['education'].isnull().sum()}")

### Comparación: Imputation inteligente vs naive

In [ ]:
naive_imputed = data_with_missing.copy()
naive_imputed['education'] = train_data['education'].mode()[0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

syn_imputed['education'].value_counts().plot(kind='bar', ax=ax1, alpha=0.8)
ax1.set_title('Imputation Inteligente (Synthetic)', fontweight='bold')
ax1.set_ylabel('Frecuencia')
ax1.tick_params(axis='x', rotation=45)

naive_imputed['education'].value_counts().plot(kind='bar', ax=ax2, alpha=0.8, color='orange')
ax2.set_title('Imputation Naive (Mode)', fontweight='bold')
ax2.set_ylabel('Frecuencia')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 La imputation inteligente preserva la variabilidad!")

### C. Rebalancing de clases

**Problema**: El dataset está desbalanceado (más personas con <=50K que con >50K)

**Solución**: Generar más registros de la clase minoritaria

In [ ]:
print("⚖️  REBALANCING: Balancear clases de income\n")

print("Distribución ORIGINAL:")
print(train_data['income'].value_counts(normalize=True))

print("\nDistribución SINTÉTICA (sin rebalancing):")
print(syn_data['income'].value_counts(normalize=True))

seed_high_income = pd.DataFrame({
    'income': ['>50K'] * 2000
})

syn_high_income = mostly.generate(g, seed=seed_high_income).data()

syn_balanced = pd.concat([syn_data, syn_high_income], ignore_index=True)

print("\nDistribución BALANCEADA:")
print(syn_balanced['income'].value_counts(normalize=True))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

train_data['income'].value_counts().plot(kind='bar', ax=axes[0], alpha=0.8)
axes[0].set_title('Original', fontweight='bold')
axes[0].set_ylabel('Frecuencia')

syn_data['income'].value_counts().plot(kind='bar', ax=axes[1], alpha=0.8)
axes[1].set_title('Sintético (sin rebalancing)', fontweight='bold')
axes[1].set_ylabel('Frecuencia')

syn_balanced['income'].value_counts().plot(kind='bar', ax=axes[2], alpha=0.8, color='green')
axes[2].set_title('Sintético (balanceado)', fontweight='bold')
axes[2].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

print("✅ Clases balanceadas para mejor entrenamiento de modelos!")

---

## 5. Datasets Multitabla con Relaciones (20 min)

En el mundo real, los datos suelen estar en múltiples tablas relacionadas.

**Ejemplo**: Separar Census en:
- **Personas**: Datos demográficos
- **Empleos**: Información laboral
- **Educación**: Historial educativo

### Crear estructura multitabla

In [ ]:
print("🏗️  CREANDO ESTRUCTURA MULTITABLA\n")

train_with_id = train_data.copy()
train_with_id['person_id'] = range(len(train_with_id))

persons = train_with_id[['person_id', 'age', 'sex', 'race', 'marital_status', 'relationship', 'native_country']].copy()

jobs = train_with_id[['person_id', 'workclass', 'occupation', 'hours_per_week', 'income']].copy()

education = train_with_id[['person_id', 'education', 'education_num']].copy()

print(f"✅ Tabla PERSONS: {persons.shape}")
print(persons.head(3))

print(f"\n✅ Tabla JOBS: {jobs.shape}")
print(jobs.head(3))

print(f"\n✅ Tabla EDUCATION: {education.shape}")
print(education.head(3))

### Configurar relaciones y entrenar

In [ ]:
multi_config = {
    'name': 'Census Multi-Table',
    'tables': [
        {
            'name': 'persons',
            'data': persons,
            'keys': [{'column': 'person_id'}]
        },
        {
            'name': 'jobs',
            'data': jobs,
            'keys': [{
                'column': 'person_id',
                'reference': {
                    'table': 'persons',
                    'column': 'person_id'
                }
            }]
        },
        {
            'name': 'education',
            'data': education,
            'keys': [{
                'column': 'person_id',
                'reference': {
                    'table': 'persons',
                    'column': 'person_id'
                }
            }]
        }
    ]
}

print("🚀 Entrenando generator multitabla...\n")
g_multi = mostly.train(multi_config)
print("\n✅ Generator multitabla entrenado!")

### Generar datos sintéticos multitabla

In [ ]:
print("🎲 Generando datos sintéticos multitabla...\n")

syn_multi = mostly.generate(g_multi, size=1000)

syn_persons = syn_multi.data('persons')
syn_jobs = syn_multi.data('jobs')
syn_education = syn_multi.data('education')

print(f"✅ Tabla PERSONS sintética: {syn_persons.shape}")
print(syn_persons.head(3))

print(f"\n✅ Tabla JOBS sintética: {syn_jobs.shape}")
print(syn_jobs.head(3))

print(f"\n✅ Tabla EDUCATION sintética: {syn_education.shape}")
print(syn_education.head(3))

### Verificar integridad referencial

In [ ]:
print("🔍 VERIFICACIÓN DE INTEGRIDAD REFERENCIAL\n")

persons_ids = set(syn_persons['person_id'])
jobs_ids = set(syn_jobs['person_id'])
education_ids = set(syn_education['person_id'])

print(f"IDs únicos en PERSONS: {len(persons_ids)}")
print(f"IDs únicos en JOBS: {len(jobs_ids)}")
print(f"IDs únicos en EDUCATION: {len(education_ids)}")

jobs_orphans = jobs_ids - persons_ids
education_orphans = education_ids - persons_ids

print(f"\n✅ Verificaciones:")
print(f"   • JOBS sin persona: {len(jobs_orphans)} (debe ser 0)")
print(f"   • EDUCATION sin persona: {len(education_orphans)} (debe ser 0)")

if len(jobs_orphans) == 0 and len(education_orphans) == 0:
    print("\n🎉 ¡Integridad referencial perfecta!")
else:
    print("\n⚠️  Hay registros huérfanos")

### Ejemplo de JOIN: Reconstruir dataset completo

In [ ]:
print("🔗 RECONSTRUYENDO DATASET COMPLETO CON JOINS\n")

syn_complete = syn_persons.merge(syn_jobs, on='person_id').merge(syn_education, on='person_id')

print(f"Dataset completo: {syn_complete.shape}")
print("\nPrimeras filas:")
print(syn_complete.head())

print("\n✅ Las relaciones se mantienen correctamente!")

### Ejemplo práctico: Análisis por JOIN

In [ ]:
print("📊 ANÁLISIS: Income por nivel educativo y género\n")

analysis = syn_complete.groupby(['education', 'sex', 'income']).size().unstack(fill_value=0)
print(analysis)

fig, ax = plt.subplots(figsize=(12, 6))
education_income = syn_complete.groupby(['education', 'income']).size().unstack()
education_income.plot(kind='bar', ax=ax, alpha=0.8)
ax.set_title('Distribución de Income por Educación (Datos Sintéticos)', fontweight='bold')
ax.set_xlabel('Nivel Educativo')
ax.set_ylabel('Frecuencia')
ax.legend(title='Income')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n✅ Podemos hacer análisis complejos manteniendo relaciones!")

### 💡 Otros casos de uso multitabla

**E-commerce:**
- Customers → Orders → Products → Reviews

**Healthcare:**
- Patients → Visits → Diagnoses → Prescriptions

**Banking:**
- Customers → Accounts → Transactions → Cards

**IoT/Logs:**
- Devices → Events → Metrics → Alerts

---

## 6. Wrap-up y Próximos Pasos (5 min)

### 🎉 ¿Qué hemos aprendido?

1. ✅ **El problema**: Anonimización tradicional no funciona
2. ✅ **La solución**: Datos sintéticos con privacidad real
3. ✅ **Generación básica**: Entrenar generators y generar datos
4. ✅ **Evaluación**: QA Reports y TSTR
5. ✅ **Control**: Seeded generation, imputation, rebalancing
6. ✅ **Multitabla**: Mantener relaciones entre entidades

### 🚀 Próximos pasos

**Recursos:**
- 📚 Documentación: [docs.mostly.ai](https://docs.mostly.ai)
- 💻 GitHub: [github.com/mostly-ai/mostlyai](https://github.com/mostly-ai/mostlyai)
- 🎓 Más ejemplos: [github.com/mostly-ai/mostly-tutorials](https://github.com/mostly-ai/mostly-tutorials)

**Funcionalidades avanzadas (no cubiertas hoy):**
- Privacidad diferencial (ε-differential privacy)
- Fairness (datos sintéticos justos)
- Datos secuenciales/temporales
- Columnas de texto (con LLMs)
- Modo cloud (más potente y rápido)

**Casos de uso:**
- 🔬 Investigación: Compartir datos sin restricciones
- 🏢 Empresas: Testing, desarrollo, demos
- 🎓 Educación: Datasets realistas para enseñanza
- 🤖 ML: Data augmentation, balancing, imputation
- 📊 Analytics: Análisis sin riesgos de privacidad

### 💬 Q&A

¿Preguntas?

---

## 🎁 Bonus: Experimentación libre

Usa las siguientes celdas para experimentar con tus propios casos:

In [ ]:
# Experimenta aquí con seeded generation


In [ ]:
# Experimenta aquí con imputation


In [ ]:
# Experimenta aquí con multitabla


---

**¡Gracias por participar!** 🙏

**Contacto:**
- Twitter/X: [@mostly_ai](https://twitter.com/mostly_ai)
- LinkedIn: [MOSTLY AI](https://www.linkedin.com/company/mostly-ai/)
- Email: support@mostly.ai

**Contribuye al proyecto:**
- ⭐ Star en GitHub
- 🐛 Reporta bugs
- 💡 Sugiere features
- 🔧 Contribuye código